In [1]:
# from google_play_scraper import Sort, reviews_all, reviews
import re
import pandas as pd
import numpy as np
import datetime as dt
import string
import pymysql
# import seaborn as sns
# import matplotlib.pyplot as plt
# import plotly.express as px
# import plotly.graph_objects as go
# from textblob import TextBlob
# from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score, recall_score, precision_score
# from sklearn.metrics import confusion_matrix, classification_report
# from google_play_scraper impo rt Sort, reviews_all, reviews
import nltk
# import calendar
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
# from wordcloud import WordCloud

print("aman")

aman


[nltk_data] Downloading package punkt to /home/idham/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
import nltk
import pandas as pd
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
import pymysql

In [7]:
def read_mysql_table(table, host='localhost', user='root', password='', database='review'):
    # Establish a connection to the MySQL database
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    
    query = f"SELECT * FROM {table}"
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Convert the result to a Pandas DataFrame
    df = pd.DataFrame(result)
    
    # Assign column names based on the cursor description
    df.columns = [column[0] for column in cursor.description]
    
    # Close the cursor and the database connection
    cursor.close()
    connection.close()
    
    return df

In [8]:
table_name = 'input_review'
df = read_mysql_table(table_name)
df.head()

,id_review,nama,tanggal,review
0,35,idham,2023-12-29,aplikasi ini keren sekali banggg


In [ ]:
# import pandas as pd
# dframe=pd.read_csv("ulasan-rating5.csv")
# print(dframe[1:4])

In [9]:
df.head()

,id_review,nama,tanggal,review
0,35,idham,2023-12-29,aplikasi ini keren sekali banggg


In [10]:
#menyimpan tweet. (tipe data series pandas)
data_content = df['review']

In [11]:
# casefolding
data_casefolding = data_content.str.lower()
data_casefolding.head()

0    aplikasi ini keren sekali banggg
Name: review, dtype: object

In [12]:
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", str(tweet)) for tweet in data_casefolding]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)


In [13]:
# #tokenize
tknzr = TweetTokenizer()
data_tokenize = [tknzr.tokenize(tweet) for tweet in data_filtering]
data_tokenize

[['aplikasi', 'ini', 'keren', 'sekali', 'banggg']]

In [14]:
#slang word
path_dataslang = open("Data/kamus kata baku-clear.csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

data_formal = []
for data in data_tokenize:
  data_clean = [replaceSlang(word) for word in data]
  data_formal.append(data_clean)
len_data_formal = len(data_formal)
# print(data_formal)
# len_data_formal

In [15]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:  
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
reviews = [removeStopWords(line,stopwords) for line in data_formal]

[nltk_data] Downloading package stopwords to /home/idham/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
reviews

[['aplikasi', 'keren', 'banggg']]

In [21]:
# Specify the file path of the pickle file
file_path = 'model_test/reviews.pkl'

# Read the pickle file
with open(file_path, 'rb') as file:
    data_train = pickle.load(file)

In [22]:
data_train

['masalah',
 'beri',
 'untuk',
 'semua',
 'game',
 'good',
 '',
 'manis',
 'apakah',
 'sumpah',
 'manis',
 'kalau',
 'pesan',
 'game',
 'baik',
 'main',
 'game',
 'saran',
 'mohon',
 'akun',
 'gamenya',
 '',
 'akhir',
 '',
 'terimakasih',
 'bagus',
 'makin',
 'terimakasih',
 'voice',
 'tolong',
 '',
 'akun',
 'hari',
 'tidak',
 'banyak',
 'fsfa',
 'coba',
 'mantap',
 'bagus',
 '',
 'game',
 '',
 '',
 'gamenya',
 'bagus',
 'saya',
 'oke',
 'sudah',
 'apakah',
 'update',
 'tidak',
 'comeback',
 'banyak',
 'tolong',
 'mantap',
 '',
 'update',
 'tolong',
 'masa',
 '',
 'game',
 'gamenya',
 'game',
 'ajar',
 'cara',
 'baik',
 'moga',
 'game',
 'untuk',
 'baru',
 '',
 'mohon',
 'saya',
 'game',
 'please',
 'developer',
 'mantap',
 'buriq',
 '',
 'game',
 'player',
 'game',
 '',
 'tolong',
 'gamenya',
 'please',
 'sayang',
 'dead',
 'mari',
 '',
 'tolong',
 'dead',
 '',
 'revamp',
 'game',
 'keren',
 'babi',
 'saran',
 '',
 'load',
 'game',
 '',
 'sangat',
 'game',
 'game',
 'terimakasih',
 '

In [23]:
# pembuatan vector kata
vectorizer = TfidfVectorizer()
train_vector = vectorizer.fit_transform(data_train)
reviews2 = [" ".join(r) for r in reviews]

In [24]:
import joblib

load_model = joblib.load(open('/home/idham/Documents/FLASK/mengajar-streamlit/model_test/revisi_hasil_sentimen.pkl','rb'))

result = []
for test in reviews2:
    test_data = [str(test)]
    test_vector = vectorizer.transform(test_data).toarray()
    pred = load_model.predict(test_vector)
    result.append(pred[0])

/usr/lib/python3.11/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [25]:
result

[5]

In [26]:
from sklearn.utils.multiclass import unique_labels
unique_labels(result)

array([5])

In [61]:
df['label'] = result

In [62]:
def delete_all_data_from_table(table, host='localhost', user='root', password='', database='review'):
    # Establish a connection to the MySQL database
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    
    # Delete all data from the specified table
    query = f"DELETE FROM {table}"
    cursor.execute(query)
    
    # Commit the changes
    connection.commit()
    
    # Close the cursor and the database connection
    cursor.close()
    connection.close()

In [63]:
delete_all_data_from_table('input_review')

In [64]:
def insert_df_into_hasil_model(df, host='localhost', user='root', password='', database='review'):
    # Establish a connection to the MySQL database
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )

    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()

    # Insert each row from the DataFrame into the 'hasil_model' table
    for index, row in df.iterrows():
        query = "INSERT INTO hasil_model (id_review, nama, tanggal, review, label) VALUES (%s, %s, %s, %s, %s)"
        cursor.execute(query, (row['id_review'], row['nama'], row['tanggal'], row['review'], row['label']))

    # Commit the changes
    connection.commit()

    # Close the cursor and the database connection
    cursor.close()
    connection.close()

In [65]:
insert_df_into_hasil_model(df)

In [66]:
table_name = 'hasil_model'
hasil_df = read_mysql_table(table_name)
hasil_df.to_csv('Data/hasil_model_baru.csv')